In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import warnings

# seaborn 사용하기
import seaborn as sns

plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

warnings.filterwarnings('ignore')

In [2]:
food = pd.read_csv('./Data/seogyodong+korean-ratio.csv')
food.head()

,기준_년_코드,기준_분기_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,점포수,한식음식점여부
0,2021,1,1000466,동교로17길,CS100001,한식음식점,243935896,14,1
1,2021,1,1000466,동교로17길,CS100002,중식음식점,45393178,1,0
2,2021,1,1000466,동교로17길,CS100003,일식음식점,55589434,3,0
3,2021,1,1000466,동교로17길,CS100006,패스트푸드점,9925407,3,0
4,2021,1,1000466,동교로17길,CS100008,분식전문점,40136157,6,0


### 붙일 데이터들 확인하기

In [3]:
# 집객시설 데이터 : 상권별로 있다. (2015-2021)
building=pd.read_csv('./Raw/서울시_우리마을가게_상권분석서비스(구_상권_집객시설).csv',encoding='euc-kr')
building.head()

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,...,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수,기준_분기_코드
0,2021,A,골목상권,1000862,방배로18길,8,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3
1,2021,A,골목상권,1000611,개봉로17다길,17,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,3
2,2021,A,골목상권,1000606,화곡로44나길,11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3
3,2021,A,골목상권,1000602,초원로16길,5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3
4,2021,A,골목상권,1000601,초록마을로8길,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3


In [11]:
building.shape

(40221, 26)

In [4]:
building.isnull().sum()

기준_년_코드           0
상권_구분_코드          0
상권_구분_코드_명        0
상권_코드             0
상권_코드_명           0
집객시설_수            0
관공서_수         28876
은행_수          25731
종합병원_수        39725
일반_병원_수       36948
약국_수          11357
유치원_수         34850
초등학교_수        38911
중학교_수         39725
고등학교_수        39823
대학교_수         39852
백화점_수         39799
슈퍼마켓_수        36855
극장_수          36402
숙박_시설_수       26658
공항_수          40221
철도_역_수        40053
버스_터미널_수      40171
지하철_역_수       36804
버스_정거장_수       8576
기준_분기_코드          0
dtype: int64

### na 값이 너무 많아서 사용이 안될 것 같다.

In [5]:
# 직장인구 수 데이터 : 상권별로 있다. (2014-2021)
pop=pd.read_csv('./Raw/서울시_우리마을가게_상권분석서비스(구_상권_직장인구).csv',encoding='euc-kr')
pop.head()

,기준_년월_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,...,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,2021,1,A,골목상권,1000007,북촌로5길,503,419,84,0,...,150,133,88,29,0,12,27,23,14,8
1,2021,1,A,골목상권,1000756,사당로8길,131,70,61,0,...,9,24,22,13,0,14,10,17,15,5
2,2021,1,R,전통시장,1001278,남평화시장,5,3,2,0,...,2,0,0,1,0,0,2,0,0,0
3,2021,1,A,골목상권,1000058,백범로90길,535,288,247,0,...,71,89,57,43,0,34,62,72,55,24
4,2021,1,A,골목상권,1000059,서빙고로71길,432,265,167,1,...,49,58,64,80,1,26,55,59,17,9


In [6]:
pop.isnull().sum()

기준_년월_코드               0
기준_분기_코드               0
상권_구분_코드               0
상권_구분_코드_명             0
상권_코드                  0
상권_코드_명                0
총_직장_인구_수              0
남성_직장_인구_수             0
여성_직장_인구_수             0
연령대_10_직장_인구_수         0
연령대_20_직장_인구_수         0
연령대_30_직장_인구_수         0
연령대_40_직장_인구_수         0
연령대_50_직장_인구_수         0
연령대_60_이상_직장_인구_수      0
남성연령대_10_직장_인구_수       0
남성연령대_20_직장_인구_수       0
남성연령대_30_직장_인구_수       0
남성연령대_40_직장_인구_수       0
남성연령대_50_직장_인구_수       0
남성연령대_60_이상_직장_인구_수    0
여성연령대_10_직장_인구_수       0
여성연령대_20_직장_인구_수       0
여성연령대_30_직장_인구_수       0
여성연령대_40_직장_인구_수       0
여성연령대_50_직장_인구_수       0
여성연령대_60_이상_직장_인구_수    0
dtype: int64

### 사용 가능할듯

In [7]:
# 디테일한 점포 수 데이터 : 업종 별로 있다.
shop=pd.read_csv('./Raw/서울시_우리마을가게_상권분석서비스(상권-점포)_2021년.csv',encoding='euc-kr')

In [8]:
shop.isnull().sum()

기준_년_코드        0
기준_분기_코드       0
상권_구분_코드       0
상권_구분_코드_명     0
상권_코드          0
상권_코드_명        0
서비스_업종_코드      0
서비스_업종_코드_명    0
점포_수           0
유사_업종_점포_수     0
개업_율           0
개업_점포_수        0
폐업_률           0
폐업_점포_수        0
프랜차이즈_점포_수     0
dtype: int64

### 사용가능할듯

In [9]:
# 아파트 데이터 : 상권 별로 (2014-2021)
apartment=pd.read_csv('./Raw/서울시_우리마을가게_상권분석서비스(구_상권_아파트).csv',encoding='euc-kr')

In [13]:
apartment.shape

(41368, 21)

In [10]:
apartment.isnull().sum()

기준_년_코드                       0
상권_구분_코드                      0
상권_구분_코드_명                    0
아파트_단지_수                      0
아파트_면적_66_제곱미터_미만_세대_수      886
아파트_면적_66_제곱미터_세대_수        3137
아파트_면적_99_제곱미터_세대_수       15287
아파트_면적_132_제곱미터_세대_수      28334
아파트_면적_165_제곱미터_세대_수      33917
아파트_가격_1_억_미만_세대_수         5100
아파트_가격_1_억_세대_수            1972
아파트_가격_2_억_세대_수            7489
아파트_가격_3_억_세대_수           20703
아파트_가격_4_억_세대_수           29019
아파트_가격_5_억_세대_수           33493
아파트_가격_6_억_이상_세대_수        35025
아파트_평균_면적                     0
아파트_평균_시가                     0
기준_분기_코드                      0
상권_코드                         0
상권_코드_명                       0
dtype: int64

### na 값 너무 많아서 사용 안될 듯

### 상권 배후지와 상권 
- 상권 배후지는 그 상권을 이용하는 고객들의 영역을 말하는 것이다. <br><br>
- 상권은 그 지역만을 말한다. <br><br>
- 예비 창업주를 대상으로 하기 때문에 배후지 파악까지는 힘들 것이라 생각하여 상권 데이터만으로 feature를 구성하려고 한다. <br><br>

### 확인해 볼만한 데이터 
- 아파트 데이터 : 아파트 평균 면적 데이터 , 아파트 단지수 ... <br><br>
- 점포수 데이터 : 프랜차이즈 점포수 데이터 <br><br>

### 나머지 데이터들로 가능한것??
- 직장인 데이터 , 상주인구 데이터, 생활인구 데이터로는 어디가 매출에 큰 비중을 차지하는지를 알아볼 수 있을 것 같다. <br><br>
- 집객시설 데이터로는 백화점이나 큰 대형마켓이 골목상권의 매출을 올려주는지 아닌지 와 상권 매출에 직접적으로 이득이 되는 집객시설이 있는지를 파악할 수 있을 것 같다. <br><br>